In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
# Load file dữ liệu sổ tay sinh viên đã được chuẩn hóa
from langchain.schema import Document
import pandas as pd

class CsvLoader(TextLoader):
    def __init__(self, file_path):
        self.df = pd.read_csv(file_path)

    def load(self):
        documents = []
        for index, row in self.df.iterrows():
            # Tạo Document từ key và value trong dictionary
            documents.append(Document(page_content=row['1'], metadata={"source": row['0']}))
        return documents
loader_data = CsvLoader("data.csv")

Phân đoạn văn bản


In [ ]:
documents_data = loader_data.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                chunk_overlap=200)

texts_data = text_splitter.split_documents(documents_data)

In [ ]:
len(texts_data)

122223

Load embedding model


In [ ]:
EMBEDDING_MODEL_PATH = r'D:\Huan\Project\KHDL\model\ebmodel\bge-m3-ft-triplet'
from langchain.embeddings import HuggingFaceBgeEmbeddings
embeddings = HuggingFaceBgeEmbeddings(model_name=EMBEDDING_MODEL_PATH)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\importlib\__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import t

Tạo vector store


In [5]:
from uuid import uuid4
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


fin = [i for i in range(2000, len(texts_data), 2000)]
fin.append(len(texts_data))

num_of_doc = 0 
start = 0 
for fi in range(len(fin)):
    i = fin[fi]
    _index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
    _uuids = [str(uuid4()) for _ in range(len(texts_data[start:i]))]
    _vector_store = FAISS(
        embedding_function=embeddings,
        index=_index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    _vector_store.add_documents(documents=texts_data[start:i], ids=_uuids)
    num_of_doc += i - start
    start = i
    _vector_store.save_local(f"faiss_index/faiss_index{fi}")
    print(f"Loaded {num_of_doc} documents and saved")

Loaded 2000 documents and saved
Loaded 4000 documents and saved
Loaded 6000 documents and saved
Loaded 8000 documents and saved
Loaded 10000 documents and saved
Loaded 12000 documents and saved
Loaded 14000 documents and saved
Loaded 16000 documents and saved
Loaded 18000 documents and saved
Loaded 20000 documents and saved
Loaded 22000 documents and saved
Loaded 24000 documents and saved
Loaded 26000 documents and saved
Loaded 28000 documents and saved
Loaded 30000 documents and saved
Loaded 32000 documents and saved
Loaded 34000 documents and saved
Loaded 36000 documents and saved
Loaded 38000 documents and saved
Loaded 40000 documents and saved
Loaded 42000 documents and saved
Loaded 44000 documents and saved
Loaded 46000 documents and saved
Loaded 48000 documents and saved
Loaded 50000 documents and saved
Loaded 52000 documents and saved
Loaded 54000 documents and saved
Loaded 56000 documents and saved
Loaded 58000 documents and saved
Loaded 60000 documents and saved
Loaded 62000 d

In [ ]:
vector_store = FAISS.load_local(
    "faiss_index/faiss_index0", embeddings, allow_dangerous_deserialization=True
) 

In [ ]:
for i in range(1, 62):
    vector_store.merge_from(FAISS.load_local(
        f"faiss_index/faiss_index{i}", embeddings, allow_dangerous_deserialization=True
    ))

In [8]:
vector_store.save_local('vector_store')

In [ ]:
docs = vector_store.similarity_search("Cách điều trị bệnh tim mạch", k = 5)

In [15]:
print(docs)

[Document(metadata={'source': 14668651}, page_content='Điều trị \nBệnh tim mạch có thể điều trị bằng điều trị ban đầu chủ yếu tập trung vào chế độ ăn uống và can thiệp lối sống. Cúm có thể làm cho các cơn đau tim và đột quỵ nhiều hơn và do đó tiêm phòng cúm có thể làm giảm nguy cơ biến cố tim mạch và tử vong ở những người mắc bệnh tim.'), Document(metadata={'source': 19826369}, page_content="Rothrock J. Alexander's - Care of the Patient. Missouri; Saunders; 2003; 236.\nComplications: p316-7 http://medind.nic.in/iad/t05/i4/iadt05i4p308.pdf Lưu trữ 2006-12-12 tại Wayback Machine\nGuidelines for cardiopulmonary resuscitation and emergency cardiovascular care. Circulation. 2000; 102 (suppl 1): I95–I104.[Medline]"), Document(metadata={'source': 470015}, page_content='Các phương pháp điều trị thay thế \nCó rất nhiều phương pháp điều trị thay thế cho cảm lạnh, hầu hết chưa có bằng chứng khoa học cụ thể nào về tác dụng của các phương pháp. và cũng chưa đủ bằng chứng cho việc khuyến cáo hay khu